# Table of Contents
* [1. Plamsa Profiles](#1.-Plamsa-Profiles)
* [2. Test Case 1. Converging Beam](#2.-Test-Case-1.-Converging-Beam)
* [3. Test Case 2. Near Cutoff Behavior](#3.-Test-Case-2.-Near-Cutoff-Behavior)
	* [3.1 Finite $k_z$](#3.1-Finite-$k_z$)
	* [3.2 Finite $k_x'/k_x^2$](#3.2-Finite-$k_x'/k_x^2$)


In [ ]:
# Some initial setups
from __future__ import print_function
import sys

import numpy as np
from scipy.integrate import trapz, cumtrapz
import numpy.fft as fft
import matplotlib.pyplot as plt
from matplotlib import rcParams

from FPSDP.GeneralSettings.UnitSystem import cgs
import FPSDP.Plasma.Analytical_Profiles.TestParameter as tp
import FPSDP.Diagnostics.ECEI.ECEI2D.Reciprocity as rcp
from FPSDP.Diagnostics.ECEI.ECEI2D.Detector2D import GaussianAntenna
import FPSDP.Plasma.PlasmaCharacter as pc


%matplotlib inline

rcParams['figure.figsize'] = [12, 9]
rcParams['font.size'] = 18

c = cgs['c']
keV = cgs['keV']
e = cgs['e']
me = cgs['m_e']
pi = np.pi
twopi = 2*np.pi

# We will use a uniform Te profile to do the benchmarks
Te0 = 10*keV
ne0 = 2e13
tp.set_parameter2D(Te_0 = Te0, ne_0=ne0, Te_shape='uniform', ne_shape='linear')
p2d_uni = tp.create_profile2D()
p2d_uni.setup_interps()

# 1. Plamsa Profiles

We have a cylindrical plasma with the following parameters:

In [ ]:
tp.show_parameter2D()

In [ ]:
print(p2d_uni.physical_quantities())

Mid-plane profiles look like:

In [ ]:
R1D = p2d_uni.grid.R1D
mid_Z = p2d_uni.grid.NZ/2

In [ ]:
plt.figure()
plt.plot(R1D, p2d_uni.ne0[mid_Z, :])
plt.title('mid-plane electron density')
plt.ylabel('ne(cm^-3)')
plt.xlabel('R(cm)')

In [ ]:
plt.figure()
plt.plot(R1D, p2d_uni.B0[mid_Z, :])
plt.title('mid-plane magnetic field')
plt.ylabel('B(Gauss)')
plt.xlabel('R(cm)')

And some relevent frequencies:

In [ ]:
pcpr = pc.PlasmaCharProfile(p2d_uni)

In [ ]:
omega = 1e11*twopi
plt.figure()
plt.plot(R1D, pcpr.omega_ce[mid_Z, :]/twopi, label='$\omega_{ce}$')
plt.plot(R1D, 2*pcpr.omega_ce[mid_Z, :]/twopi, label='$2\omega_{ce}$')
plt.plot(R1D, 3*pcpr.omega_ce[mid_Z, :]/twopi, label='$3\omega_{ce}$')
plt.plot(R1D, pcpr.omega_pe[mid_Z, :]/twopi, label='$\omega_{pe}$')
plt.plot(R1D, pcpr.omega_R[mid_Z, :]/twopi, label='$R \; cutoff$')
plt.plot(R1D, pcpr.omega_UH[mid_Z, :]/twopi, label='$\omega_{UH}$')
plt.hlines(y=omega/twopi, xmin=150, xmax=300)
plt.legend(loc='best')
plt.ylabel('frequency (Hz)')
plt.xlabel('R (cm)')

# 2. Test Case 1. Converging Beam

Strongly converging beams are of particular interests because they have narrow waist, so may provide good resolution in Y and Z directions. We will use a Gaussian beam with waist width equals 2 times vacuum wavelength:
$$w_{0y} = w_{z0} = 2\lambda$$

In [ ]:
omega = twopi*1e11
k = omega/c
wave_length = twopi/k
print('lambda = {0:.3}cm'.format(wave_length))

In [ ]:
detector = GaussianAntenna(omega_list=[omega], k_list=[k], power_list=[1], waist_x=220, waist_y=0, w_0y=2*wave_length)

In [ ]:
detector.central_beam.reighlay_range

In [ ]:
detector.central_beam.divergence

In [ ]:
ece = rcp.ECE2D(p2d_uni, detector, polarization='X', max_harmonic=4, max_power=4, 
                weakly_relativistic=True, isotropic=True)

In [ ]:
X1D = np.linspace(251, 210, 100)
Y1D = np.linspace(-20, 20, 129)
Z1D = np.linspace(-20, 20, 129)

In [ ]:
ece.set_coords([Z1D, Y1D, X1D])

In [ ]:
ece.auto_adjust_mesh()

In [ ]:
ece.view_point

In [ ]:
ece.diag_x

In [ ]:
Te = ece.diagnose()

In [ ]:
Te/keV

In [ ]:
plt.contour(ece.X1D, ece.Y1D, ece.view_spot, np.linspace(0.5,1.5,5)*0.368*np.max(ece.view_spot))
plt.title('View spot')
plt.xlabel('X(cm)')
plt.ylabel('Y(cm)')

In [ ]:
E = ece.propagator.E[0,:,::2]
plt.contour(ece.X1D, ece.Y1D, np.abs(E), np.linspace(0.5,1.5,5)*0.368*np.max(np.abs(E)))
plt.vlines(x=220, ymin=-3*wave_length, ymax=3*wave_length)
plt.title('Electric field amplitude')
plt.ylabel('Y(cm)')
plt.xlabel('X(cm)')

In [ ]:
pf = ece.propagator.power_flow
plt.plot(ece.propagator.calc_x_coords, np.abs(pf))
plt.title('power flow of the wave')

# 3. Test Case 2. Near Cutoff Behavior

When electron density increases, the R-cutoff frequency and electron plasma frequency will rise. They corresponds to X-mode and O-mode cutoff respectively. When waves approaches cutoff, $k_x \to 0$, some of the assumptions made in our paraxial propagator will be invalid. Namely, they are:
- Paraxial assumption: $|k_y/k_x| \ll 1 $ and $|k_z/k_x|\ll 1$. The boundary where this assumption starts to fail depends on the incidental angle of the main ray, as well as the spread of the whole beam. 
- WKB assumption: $|k_x'/k_x^2| \ll 1$. For a linear density profile, let $L$ be the gradient length scale at the cutoff location, i.e. $L = n_c/\nabla n$. We can show that this criteria reduces to the requirement $x \gg \sqrt{\lambda_0 L/2\pi}$, where $x$ is the distance from cutoff, and $\lambda_0$ is the wave length in vacuum.

In [ ]:
# We use a higher electron density to investigate the near cutoff behavior of our propagator and ECE2D modules
ne0_high = 8e13
tp.set_parameter1D(Te_0 = Te0, ne_0=ne0_high, Te_shape='uniform', 
                   ne_shape='linear', R_0=1000, Xmin=950, Xmax=1100)
p1d_high = tp.create_profile1D()

from FPSDP.Geometry.Grid import Cartesian2D
grid_h = Cartesian2D(DownLeft=(-50, 950), UpRight=(50, 1100), NR=301, NZ=201)
mid_Zh = grid_h.NZ/2
p2d_high = tp.simulate_1D(p1d=p1d_high, grid2D=grid_h)
p2d_high.setup_interps()

In [ ]:
pcpr_h = pc.PlasmaCharProfile(p2d_high)

In [ ]:
omega = 0.8e11*twopi
k = omega/c
wave_length = twopi/k
R1D = p2d_high.grid.R1D
plt.figure()
plt.plot(R1D, pcpr_h.omega_ce[mid_Zh, :]/twopi, label='$\omega_{ce}$')
plt.plot(R1D, 2*pcpr_h.omega_ce[mid_Zh, :]/twopi, label='$2\omega_{ce}$')
plt.plot(R1D, 3*pcpr_h.omega_ce[mid_Zh, :]/twopi, label='$3\omega_{ce}$')
plt.plot(R1D, pcpr_h.omega_pe[mid_Zh, :]/twopi, label='$\omega_{pe}$')
plt.plot(R1D, pcpr_h.omega_R[mid_Zh, :]/twopi, label='$R \; cutoff$')
plt.plot(R1D, pcpr_h.omega_UH[mid_Zh, :]/twopi, label='$\omega_{UH}$')
plt.hlines(y=omega/twopi, xmin=np.min(R1D), xmax=np.max(R1D))
plt.legend(loc='best')
plt.ylabel('frequency (Hz)')
plt.xlabel('R (cm)')

## 3.1 Finite $k_z$

We will use a weakly diverging beam with a significant horizontal incident angle. So central $k_z$ is finite, but k_z spread is small.  

In [ ]:
theta_h = pi/10

In [ ]:
detector = GaussianAntenna(omega_list=[omega], k_list=[k], power_list=[1], waist_x=1050, 
                           waist_y=0, w_0y=10*wave_length, tilt_h=theta_h)

In [ ]:
k_z_central = k*np.sin(theta_h)
print('central k_z:', k_z_central)

Due to our choice of omega and B field profile, we are far away from ECE resonance. We can focus on the wave propagation pattern.

In [ ]:
ece_h = rcp.ECE2D(p2d_high, detector, polarization='X', max_harmonic=2, max_power=2, 
                weakly_relativistic=True, isotropic=True)

In [ ]:
X1D = np.linspace(1051, 1034, 200)
Y1D = np.linspace(-20, 20, 129)
Z1D = np.linspace(-10, 20, 257)

In [ ]:
ece_h.set_coords([Z1D, Y1D, X1D])

In [ ]:
ece_h.diagnose()

In [ ]:
ece_h.Te/keV

In [ ]:
prop = ece_h.propagator
E = np.fft.ifft(prop.E, axis=0)

In [ ]:
fig = plt.figure()
plt.contour(prop.calc_x_coords, prop.z_coords, np.abs(E[:,64,:]), 30)
fig.axes[0].set_aspect(1)
plt.vlines(x=[1041], ymin=-10, ymax=20)
plt.locator_params(axis='x', nbins=5)
plt.title('abs(E0)')
plt.ylabel('Z(cm)')
plt.xlabel('X(cm)')

In [ ]:
ece_h.propagator._generate_main_phase()
main_phase = ece_h.propagator.main_phase

In [ ]:
plt.contour(prop.x_coords, prop.z_coords, np.angle(E[:,64,:]*np.exp(1j*main_phase)), 30)
plt.title('Wave front shape')

> Questions
- **Why is there a tilde at the edge?**
- **Why was the phase variation get larger near the cutoff?**

If we look at the main wave vector.

In [ ]:
print('central_kz:{0:.3}, delta_kz:{1:.3}'.format(prop.central_kz[0,0], prop.delta_kz[0,0]))

In [ ]:
plt.plot(prop.calc_x_coords, np.abs(prop.k_0)/k_z_central)
plt.hlines(y=[1,2], xmin=1034, xmax=1051)
plt.title('$k_0$ plot')
plt.ylabel('$k_0/k_z$')
plt.xlabel('X(cm)')

In [ ]:
pf = prop.power_flow
plt.plot(prop.calc_x_coords, np.abs(pf))
plt.title('power flow of the wave')

> Questions
- **Why does the power flow start with 1.05 instead of 1?**
- **Why does it decrease?**

## 3.2 Finite $k_x'/k_x^2$

Using the settings above, we propagate a perpendicularly incident wave instead, and take a look at the $k_x'/k_x^2$ term effect.

In [ ]:
detector = GaussianAntenna(omega_list=[omega], k_list=[k], power_list=[1], waist_x=1050, 
                           waist_y=0, w_0y=20*wave_length, tilt_h=0)

In [ ]:
ece_p = rcp.ECE2D(p2d_high, detector, polarization='X', max_harmonic=2, max_power=2, 
                weakly_relativistic=True, isotropic=True)

In [ ]:
X1D = np.linspace(1051, 1034, 300)
Y1D = np.linspace(-20, 20, 129)
Z1D = np.linspace(-20, 20, 129)

In [ ]:
ece_p.set_coords([Z1D, Y1D, X1D])

In [ ]:
ece_p.diagnose()

In [ ]:
ece_p.Te/keV

In [ ]:
prop = ece_p.propagator
E = np.fft.ifft(prop.E, axis=0)

In [ ]:
fig = plt.figure()
plt.contour(prop.calc_x_coords, prop.z_coords, np.abs(E[:,64,:]), 30)
plt.vlines(x=[1035], ymin=-20, ymax=20)
fig.axes[0].set_aspect(1)
plt.locator_params(axis='x', nbins=5)
plt.title('abs(E0)')
plt.ylabel('Z(cm)')
plt.xlabel('X(cm)')

In [ ]:
prop._generate_main_phase()
main_phase = prop.main_phase

In [ ]:
plt.plot(prop.calc_x_coords, main_phase)

In [ ]:
plt.contour(prop.calc_x_coords, prop.z_coords, np.angle(E[:,64,:]*np.exp(1j*main_phase)), 30)
plt.title('Wave front shape')

In [ ]:
k0 = np.abs(prop.k_0)
dx = np.abs(prop.calc_x_coords[1] - prop.calc_x_coords[0])
kp = (k0[2:]-k0[:-2])/(2*dx)
WKB_term = np.abs(kp/(k0[1:-1]*k0[1:-1]))
plt.plot(prop.calc_x_coords[1:-1], WKB_term)
plt.hlines(y=[1, 0.1], xmin=1034, xmax=1051)
plt.title('WKB criteria')
plt.ylabel('$k_x\'/k_x^2$')

In [ ]:
lambd = twopi*c/omega
np.sqrt(16*lambd/twopi)

The area where WKB approximation fails roughly agrees with our estimation. 

In [ ]:
pf = prop.power_flow
plt.plot(prop.calc_x_coords, np.abs(pf))
plt.title('power flow of the wave')